# Importar os módulos necessários

In [193]:
import warnings
warnings.filterwarnings("ignore")

In [194]:
import pandas as pd
#necessario por causa do metodo de avaliaçao
from sklearn.model_selection import KFold
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier # decision trees for classification
from sklearn.neural_network import  MLPClassifier # neural networks for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVC # support vector machines for classification

# Definir as Métricas para Avaliação dos Modelos

In [195]:
#metricas regressao
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def custom_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return mse, mae, r2


In [196]:
#metricas classificação
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [197]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import make_scorer
import numpy as np

# Função para calcular o RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Dicionário de métricas de regressão
REGRESSION_METRICS = {
    "MSE": make_scorer(mean_squared_error),
    "MAE": make_scorer(mean_absolute_error),
    "R2": make_scorer(r2_score),
    "RMSE": make_scorer(root_mean_squared_error)  # Adicionando o RMSE
}


In [198]:
METRICS = {
        "accuracy": make_scorer(accuracy_score),
        "precision": make_scorer(precision_score),
        "recall": make_scorer(recall_score),
        "f1": make_scorer(f1_score),
        "AUC": make_scorer(roc_auc_score, needs_proba=True),
        "specificity": make_scorer(specificity_score),
        "kappa":make_scorer(cohen_kappa_score)
}

# Ler o Conjunto de Dados

In [199]:
d = pd.read_csv("Acenario2Ideia2_16_12.csv")
d

,id_transmissao,id_body_type,id_modelos,mileage,ano,id_city,id_insurance,seats,engine_capacity,id_fuel_type,kms_driven,max_power,id_owner_type,resale_price_Lakh
0,0.142984,0.374130,3.103368,23.84000,2019.0,1.089000,0.259343,5.0,1199.0,0.186675,30910.0,83.8,0.154422,5.66
1,0.142984,0.596986,2.420485,17.50000,2018.0,1.289242,1.321287,5.0,1199.0,0.186675,48089.0,88.7,1.347493,6.64
2,0.142984,0.562130,2.588458,17.40000,2015.0,0.918161,0.259343,5.0,1497.0,0.186675,51000.0,117.3,0.617518,5.65
3,0.551807,0.596986,3.667640,14.08000,2021.0,0.918161,0.259343,7.0,1956.0,0.500914,30000.0,167.6,0.154422,23.00
4,0.551807,0.374130,2.764550,21.40000,2019.0,1.173949,1.321287,5.0,1197.0,0.186675,61113.0,83.1,0.154422,6.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13951,0.551807,1.362288,3.667640,19.49405,2021.0,1.033162,0.259343,7.0,2199.0,0.500914,80000.0,197.2,0.617518,26.50
13952,0.142984,0.374130,2.968670,19.49405,2017.0,1.221517,0.259343,5.0,1197.0,0.186675,17923.0,81.8,0.154422,5.87
13953,0.142984,0.562130,3.299663,19.49405,2017.0,1.033162,0.439582,5.0,1498.0,0.500914,63389.0,108.6,0.617518,7.43
13954,0.142984,0.596986,2.764550,19.49405,2017.0,0.870141,0.439582,5.0,1248.0,0.500914,40000.0,88.5,0.617518,9.45


In [200]:
X, y = d.drop("resale_price_Lakh", axis=1), d["resale_price_Lakh"]

# Definir o Método de Validação Cruzada

In [201]:
#Classificação
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)


# Implementação dos Algoritmos de Machine Learning

### Árvores de Decisão

In [202]:
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Definir o regressor e o método de validação cruzada
dt_regressor = DecisionTreeRegressor(max_depth=10, random_state=1234)
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = {k: v.mean() for k, v in scores.items()}  # Negar os valores para tornar os scores positivos
pd.DataFrame(dt_scores, index=[0])



,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.172993,0.009924,19.98603,1.624352,0.842587,4.37452


In [203]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Continue com a validação cruzada e a avaliação de métricas
dt_regressor = DecisionTreeRegressor(max_depth=7, random_state=1234)
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T





,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.114998,0.006302,19.84385,1.833215,0.839607,4.410908


In [204]:
dt = DecisionTreeRegressor(max_depth=3, random_state=1234)
splitter = KFold(10, random_state=1234, shuffle=True)
scores = cross_validate(dt, X, y, cv=splitter, scoring=REGRESSION_METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T

,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.031911,0.005197,30.304009,2.950834,0.752306,5.467498


In [205]:


# Criar um modelo de regressão de árvore de decisão
dt_regressor = DecisionTreeRegressor(max_depth=3, random_state=1234)

# Estratégia de validação cruzada (KFold) para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Continuar com a validação cruzada e avaliação de métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T



,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_AUC,test_specificity,test_kappa
0,0.035467,0.015381,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Redes Neuronais

In [206]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

# Criar uma Rede Neural para regressão
nn_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=40, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_nn = cross_validate(nn_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,8.679027,0.007432,145.584861,7.18615,-0.031939,10.222328


In [207]:
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Gerar dados de exemplo
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

# Criar um regressor de floresta aleatória
rf_regressor = RandomForestRegressor(max_depth=7, random_state=0)



# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=5, scoring=REGRESSION_METRICS)
rf_scores = pd.DataFrame(scores_rf)
print(pd.DataFrame(rf_scores.mean()).T)
 


   fit_time  score_time   test_MSE  test_MAE   test_R2  test_RMSE
0  0.412954    0.018446  96.984074  7.434062  0.932186   9.647133


### Naive Bayes

### Support Vector Machine